In [0]:
!wget -q -nc https://raw.githubusercontent.com/pseprivamirakbarnejad/cmput206lab/master/Lab8/circle.png
!ls

circle.png  sample_data


In [0]:
import numpy as np
import cv2
from sklearn.cluster import KMeans
from skimage.color import gray2rgb
import scipy.ndimage.filters as filters
from matplotlib import pyplot as plt

# Compute accumulator for circle detection

To simplify the problem, both circles have radius *`r = 64`* so you only need to find their centers *`(cx, cy)`*

1. choose suitable minimum and maximum values for *`cx`* and *`cy`* along with the number of values *`num_cx`* and *`num_cy`* to sample the respective intervals.
2. create vectors *`cx_vec`* and *`cy_vec`* of sizes *`num_cx`* and *`num_cy`* respectively containing all possible values of *`cx`* and *`cy`* (hint: *`np.linspace`*)
3. create the accumulator array of size *`num_cx`* $\times$ *`num_cy`* containing all zeros and of type *uint64*
4. find locations of all edge pixels in the input image (hint: *`np.nonzero`*)
5. iterate over the edge pixels and fill up the accumulator array

  5.1. for each edge pixel *`x, y`*, iterate over all values of *`cx`* and calculate the *`cy`* corresponding to each *`cx`* using the equation for circle:

  $(x - cx)^2 + (y - cy)^2 = r^2$

  where  *`r = 64`*

  5.2. find the index of this `cy` within `cy_vec` which will give the column index for accumulator while the index of *`cx`* within *`cx_vec`* gives the corresponding row index.

  5.3. increment the accumulator entry corresponding to this row and column index by 1
6. The filled accumulator array should look similar to this:
<img src="https://raw.githubusercontent.com/pseprivamirakbarnejad/cmput206lab/master/Lab8/circle_accumulator.png">



In [0]:
def hough_circle(img, r):
  """
  add your code here
  """
  cx_min = 0
  cy_min = 0
  cx_max = 250
  cy_max = 250
  num_cx = 250
  num_cy = 250
  cx_vec = np.linspace (cx_min,cx_max,num_cx)
  cy_vec = np.linspace (cy_min,cy_max,num_cy)
  accumulator = np.zeros((num_cx,num_cy),dtype=np.uint64)
  y_idxs, x_idxs  = np.nonzero(img)
  for i in range(len(x_idxs)):
    x = x_idxs[i]
    y = y_idxs[i]
    for cx_idx in range(num_cx):
      cx = cx_vec[cx_idx]
      cy = y - np.sqrt(np.absolute(np.square(r) - np.square(x - cx)))
      cy_idx = int((cy-cy_min)/(cy_max-cy_min)*(num_cy-1))
      accumulator[cx_idx,cy_idx] += 1
  return accumulator, cx_vec, cy_vec

# Find the locations of two distinct local maxima in the accumulator array

1. choose a suitable integer *`n`* such that top *`n`* maxima will include pixels from both maxima

2. find the top *`n`* maxima (hint: *`np.argsort`*) and fill up *`max_idx`* which is *`2 x n`* array containing the *`x, y`* coordinates of these *n* maxima. A binary image showing the maxima may look like this:
<img src="https://raw.githubusercontent.com/pseprivamirakbarnejad/cmput206lab/master/Lab8/circle_maxima.png">

3. use KMeans clustering to find 2 clusters among these *`n`* points:

    https://towardsdatascience.com/machine-learning-algorithms-part-9-k-means-example-in-python-f2ad05ed5203

4. obtain the centers of the two clusters (hint: *`kmeans.cluster_centers_`*) that correspond to the indices of the two *`cx, cy`* pairs

5. extract the corresponding *`cx, cy`* values from *`cx_vec`* and *`cy_vec`*

6. the detected circles when drawn on the input image should closely overlay the source circles:

<img src="https://raw.githubusercontent.com/pseprivamirakbarnejad/cmput206lab/master/Lab8/circle_detections.png">


In [0]:
def find_maxima(accumulator, cx_vec, cy_vec):
  """
  add your code here
  """
  idx = np.argsort(accumulator)
  n = accumulator[idx]
  max_idx = np.array((n,n))
  kmeans = KMeans(n_clusters=2, random_state=0).fit(max_idx)
  (cx1_idx,cy1_idx),(cx2_idx,cy2_idx) = kmeans.cluster_centers_[:,0],kmeans.cluster_centers_[:,1]
  cx1 = cx_vec[cx1_idx]
  cy1 = cy_vec[cy1_idx]
  cx2 = cx_vec[cx2_idx]
  cy2 = cy_vec[cy2_idx]

  maxima_img = np.zeros_like(accumulator).astype(np.uint8)
  for (x, y) in max_idx:
      maxima_img[x, y] = 255

  plt.figure()
  plt.title('Accumulator Maxima')
  plt.imshow(maxima_img, cmap='gray')
  plt.axis('off')

  return cx1, cy1, cx2, cy2

**Do not change anything in the following code**

In [0]:
def draw_circle(t_vec, num_t, r, cx, cy, w, h, out_img, col):
    for t_idx in range(num_t):
        t = t_vec[t_idx]

        t = np.deg2rad(t)

        x = r * np.cos(t) + cx
        y = r * np.sin(t) + cy

        if x > 0 and y > 0 and x < w and y < h:
            out_img[int(y), int(x)] = col

In [0]:
# circle radius
r = 64

img = cv2.imread('circle.png', 0)
accumulator, cx_vec, cy_vec = hough_circle(img, r)

accumulator_img = accumulator.astype(np.float32) / np.amax(accumulator)

cx1, cy1, cx2, cy2 = find_maxima(accumulator, cx_vec, cy_vec)

h, w = img.shape

t_min, t_max, num_t = 0, 360, 720
t_vec = np.linspace(t_min, t_max, num_t)

detections_img = gray2rgb(img)
draw_circle(t_vec, num_t, r, cx1, cy1, w, h, detections_img, (0, 255, 0))
draw_circle(t_vec, num_t, r, cx2, cy2, w, h, detections_img, (0, 255, 0))


plt.figure()
plt.title('Input image')
plt.imshow(img, cmap='gray')
plt.axis('off')

plt.figure()
plt.title('Accumulator')
plt.imshow(accumulator_img, cmap='gray')
plt.axis('off')

plt.figure()
plt.title('Detected Circles')
plt.imshow(detections_img, cmap='gray')
plt.axis('off')

ValueError: ignored